### **This notebook generates segmentation results for validation data with input size 240*240**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install nilearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation ,Dropout
from keras.layers.convolutional import Conv2D, UpSampling2D,Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from sklearn.model_selection import train_test_split
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
from keras import backend as K
import glob
import skimage.io as io
import skimage.color as color
import random as r
import math
from nilearn import plotting
import pickle
import skimage.transform as skTrans
from nilearn import image
from nilearn.image import resample_img
import nibabel.processing
import warnings
import shutil

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### **Data Preprocessing**

In [ ]:
def Data_Preprocessing(modalities_dir):
    all_modalities = []    
    for modality in modalities_dir:   
        nifti_file   = nib.load(modality)
        brain_numpy  = np.asarray(nifti_file.dataobj)    
        all_modalities.append(brain_numpy)
    all_modalities = np.array(all_modalities)
    all_modalities = np.rint(all_modalities).astype(np.int16)
    all_modalities = all_modalities[:, :, :, :]
    all_modalities = np.transpose(all_modalities)
    avg_modality=[]
    for i in range(len(all_modalities)):
      x=(all_modalities[i,:,:,0]+all_modalities[i,:,:,1]+all_modalities[i,:,:,2]+all_modalities[i,:,:,3])/4
      avg_modality.append(x)  
    gt=all_modalities[:,:,:,4]
    P_Data=np.stack(np.stack((avg_modality, gt), axis = -1))
    #P_Data=np.stack(np.stack((avg_modality), axis = -1))#for validation dataset 
    
    

    return P_Data

In [ ]:
batch=120

In [ ]:
Path='/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'# for validation dataset 
p=os.listdir(Path)
Input_Data= []

# generate data in batches or else computationl resources get exhausted
for i in p[batch-40:batch+5]:
  
    brain_dir = os.path.normpath(Path+'/'+i)
    flair     = glob.glob(os.path.join(brain_dir, '*_flair*.nii'))
    t1        = glob.glob(os.path.join(brain_dir, '*_t1*.nii'))
    t1ce      = glob.glob(os.path.join(brain_dir, '*_t1ce*.nii'))
    t2        = glob.glob(os.path.join(brain_dir, '*_t2*.nii'))
    gt        = glob.glob( os.path.join(brain_dir, '*_seg*.nii'))
    
    modalities_dir = [flair[0], t1[0], t1ce[0], t2[0], gt[0]]
    
    P_Data = Data_Preprocessing(modalities_dir)
    Input_Data.append(P_Data)
    print('This is done ', i)

This is done  BraTS20_Validation_081
This is done  BraTS20_Validation_082
This is done  BraTS20_Validation_084
This is done  BraTS20_Validation_083
This is done  BraTS20_Validation_085
This is done  BraTS20_Validation_086
This is done  BraTS20_Validation_087
This is done  BraTS20_Validation_088
This is done  BraTS20_Validation_089
This is done  BraTS20_Validation_090
This is done  BraTS20_Validation_091
This is done  BraTS20_Validation_092
This is done  BraTS20_Validation_093
This is done  BraTS20_Validation_095
This is done  BraTS20_Validation_094
This is done  BraTS20_Validation_096
This is done  BraTS20_Validation_097
This is done  BraTS20_Validation_098
This is done  BraTS20_Validation_099
This is done  BraTS20_Validation_100
This is done  BraTS20_Validation_101
This is done  BraTS20_Validation_102
This is done  BraTS20_Validation_103
This is done  BraTS20_Validation_104
This is done  BraTS20_Validation_105
This is done  BraTS20_Validation_106
This is done  BraTS20_Validation_107
T

### **Generating Segmentation Results**

In [ ]:
def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x

def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)
    maxp_1 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_1)
    
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_2)
    
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_3)
    
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_4)
    
    conv_5 = Convolution(maxp_4,512)
    upsample_6 = UpSampling2D((2, 2)) (conv_5)
    
    conv_6 = Convolution(upsample_6,256)
    upsample_7 = UpSampling2D((2, 2)) (conv_6)
    
    upsample_7 = concatenate([upsample_7, conv_3])
    
    conv_7 = Convolution(upsample_7,128)
    upsample_8 = UpSampling2D((2, 2)) (conv_7)
    
    conv_8 = Convolution(upsample_8,64)
    upsample_9 = UpSampling2D((2, 2)) (conv_8)
    
    upsample_9 = concatenate([upsample_9, conv_1])
    
    conv_9 = Convolution(upsample_9,32)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv_9)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model

In [ ]:
model = model(input_shape = (240,240,1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 240, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 240, 240, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 240, 240, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 240, 240, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model.load_weights('/content/drive/MyDrive/MRI Data/Brats2020_20Images/BraTs2020_20.h5')

In [ ]:
def Data_Concatenate(Input_Data):
    counter=0
    Output= []
    for i in range(2):
        print('$')
        c=0
        counter=0
        for ii in range(len(Input_Data)):

            if (counter < len(Input_Data)-1):
                a= Input_Data[counter][:,:,:,i]
                #print('a={}'.format(a.shape))
                b= Input_Data[counter+1][:,:,:,i]
                #print('b={}'.format(b.shape))
                if (counter==0):
                    c= np.concatenate((a, b), axis=0)
                    #print('c1={}'.format(c.shape))
                    counter= counter+2
                else:
                    c1= np.concatenate((a, b), axis=0)
                    c= np.concatenate((c, c1), axis=0)
                    print('c2={}'.format(c.shape))
                    counter= counter+2
            
            if (counter == len(Input_Data)-1):
              a= Input_Data[counter][:,:,:,i]
              c= np.concatenate((c, a), axis=0)
              print('c2={}'.format(c.shape))
              counter=counter+2

        print('c2={}'.format(c.shape))
        c= c[:,:,:,np.newaxis]
        Output.append(c)
    return Output

In [ ]:
InData= Data_Concatenate(Input_Data)

$
c2=(620, 240, 240)
c2=(930, 240, 240)
c2=(1240, 240, 240)
c2=(1550, 240, 240)
c2=(1860, 240, 240)
c2=(2170, 240, 240)
c2=(2480, 240, 240)
c2=(2790, 240, 240)
c2=(3100, 240, 240)
c2=(3410, 240, 240)
c2=(3720, 240, 240)
c2=(4030, 240, 240)
c2=(4340, 240, 240)
c2=(4650, 240, 240)
c2=(4960, 240, 240)
c2=(5270, 240, 240)
c2=(5580, 240, 240)
c2=(5890, 240, 240)
c2=(6200, 240, 240)
c2=(6510, 240, 240)
c2=(6820, 240, 240)
c2=(6975, 240, 240)
c2=(6975, 240, 240)
$
c2=(620, 240, 240)
c2=(930, 240, 240)
c2=(1240, 240, 240)
c2=(1550, 240, 240)
c2=(1860, 240, 240)
c2=(2170, 240, 240)
c2=(2480, 240, 240)
c2=(2790, 240, 240)
c2=(3100, 240, 240)
c2=(3410, 240, 240)
c2=(3720, 240, 240)
c2=(4030, 240, 240)
c2=(4340, 240, 240)
c2=(4650, 240, 240)
c2=(4960, 240, 240)
c2=(5270, 240, 240)
c2=(5580, 240, 240)
c2=(5890, 240, 240)
c2=(6200, 240, 240)
c2=(6510, 240, 240)
c2=(6820, 240, 240)
c2=(6975, 240, 240)
c2=(6975, 240, 240)


In [ ]:
AIO= concatenate(InData, axis=3)
AIO=np.array(AIO,dtype='float32')
TR=np.array(AIO[:,:,:,0],dtype='float32')
TRL=np.array(AIO[:,:,:,1],dtype='float32')#segmentation
AIO=TRL=0
AIO=np.array(AIO,dtype='float32')

In [ ]:
#predict segmentation
Segmentation = model.predict(TR)


In [ ]:
# save segmentation results for further use
with open('/content/drive/MyDrive/MRI Data/Brats2020_20Images/Segmentation_Output_80_125'+'.pkl','wb') as f:
    pickle.dump(Segmentation,f)

### **Storing Segmentaion Results in Google Drive**

In [ ]:
#converting Segmentation result back to 125 sections
Section=[]
previous=0
for i in range(len(Segmentation)):
  if (i % 155 == 0):
    a=Segmentation[i:i+155,:,:,0]
    Section.append(a)  

In [ ]:
Section = np.transpose(Section, (0,3,2,1))

In [ ]:
x=nib.load('/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_flair.nii')
target_affine=x.affine

In [ ]:
# generate .nii.gz files
counter=81
for i in range(len(Section)):
 
  data=Section[i]
  img = nib.Nifti1Image(data, target_affine)
  nib.save(img, '/content/drive/MyDrive/MRI Data/Brats2020_20Images/BraTS20_Validation_'+str(counter).zfill(3)+'.nii.gz')  
  print('/content/drive/MyDrive/MRI Data/Brats2020_20Images/BraTS20_Validation_'+str(counter).zfill(3)+'.nii.gz')
  counter=counter+1